In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant

# from imblearn.over_sampling import SMOTE
# from collections import Counter

# fixed-effect Bayes model

In [7]:
data_smo = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/smote/smote_data.csv")
data_train_smo = data_smo[data_smo.target == "train"]
data_train_smo.reset_index(inplace = True)

In [8]:
data_test_smo = data_smo[data_smo.target == "test"]
data_test_smo.reset_index(inplace = True)

In [9]:
data_raw = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/output/data_valid.csv")

data_valid = data_raw
i = 4
data_valid = data_valid[data_valid.cycle <= i]

data_test = data_valid[data_valid["visit_year"] > 2018]
data_test.reset_index(inplace = True)
X_test = data_test[[x for x in data.columns if x not in ["patient_sn", "visit_year", "cycle_num", "event"]]]
y_test = data_test["event"]

## prior model

In [10]:
# train the bayes model with static features
prior_feature_list = ["gender", "tumor_stage_pN", "cycle"]
clf_prior = CategoricalNB()
clf_prior_fit = clf_prior.fit(data_train_smo[prior_feature_list], data_train_smo["event"])

# output prior for next model
data_train_smo["prior"] = clf_prior_fit.predict_proba(data_train_smo[prior_feature_list])[:,1]
data_test_smo["prior"] = clf_prior_fit.predict_proba(data_test_smo[prior_feature_list])[:,1]


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/3223399617.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_smo["prior"] = clf_prior_fit.predict_proba(data_train_smo[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/3223399617.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_smo["prior"] = clf_prior_fit.predict_proba(data_test_smo[prior_feature_list])[:,1]


## combined model in SMO-data

In [11]:
quartile_list = ["PLT_q", "Hb_q", "WBC_q", "Neut_q", "Lymph_q"]

clf_quartile = GaussianNB(priors = [0.5, 0.5])  
clf_quartile_fit = clf_quartile.fit(data_train_smo[quartile_list], data_train_smo["event"])

In [15]:
def predict_by_model(df):
    df["prior"] = clf_prior_fit.predict_proba(df[prior_feature_list])[:,1]
    df["prior_odds"] = df["prior"]/(1 - df["prior"])

    quartile_pre = clf_quartile_fit.predict_proba(df[quartile_list])[:,1]
    df["quartile_odds"] = quartile_pre/(1- quartile_pre)
    df["posterior_odds"] = df["quartile_odds"]*df["prior_odds"]
    df["posterior"] = df["posterior_odds"]/(1+df["posterior_odds"])  

    return df

In [16]:
data_train_smo  = predict_by_model(data_train_smo)

fpr, tpr, thresholds = roc_curve(data_train_smo["event"], data_train_smo["posterior"])
print(len(data_train_smo["event"]))
print(auc(fpr, tpr))

# data_train_smo.to_csv("./output/smote_model_smote_trainset_with_prediction.csv", index=False)

3870
0.7927675286608042


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_fit.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:6: SettingWithCopyWarning: 
A value is trying to be set on a

In [17]:
data_test_smo  = predict_by_model(data_test_smo)

print(len(data_test_smo["event"]))
fpr, tpr, thresholds = roc_curve(data_test_smo["event"], data_test_smo["posterior"])
print(auc(fpr, tpr))

# data_test_smo.to_csv("./output/smote_model_smote_testset_with_prediction.csv", index=False)

790
0.8882006088767824


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_fit.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/519203610.py:6: SettingWithCopyWarning: 
A value is trying to be set on a

## prior model（probability calibrated by isotonic） 

In [18]:
# train the bayes model with static features
prior_feature_list = ["gender", "tumor_stage_pN", "cycle"]
clf_prior = CategoricalNB()
# clf_prior = MultinomialNB()
# clf_prior = GaussianNB()
# clf_prior = ComplementNB()

# probability calibrated（method="isotonic")
clf_prior_isotonic = CalibratedClassifierCV(clf_prior, cv=2, method="isotonic")
clf_prior_isotonic_fit = clf_prior_isotonic.fit(data_train_smo[prior_feature_list], data_train_smo["event"])

# output prior 
data_train_smo["prior"] = clf_prior_isotonic_fit.predict_proba(data_train_smo[prior_feature_list])[:,1]
data_test_smo["prior"] = clf_prior_isotonic_fit.predict_proba(data_test_smo[prior_feature_list])[:,1]

/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/855982914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_smo["prior"] = clf_prior_isotonic_fit.predict_proba(data_train_smo[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/855982914.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_smo["prior"] = clf_prior_isotonic_fit.predict_proba(data_test_smo[prior_feature_list])[:,1]


## prior model（probability calibrated by isotonic）in SMO-data

In [19]:
quartile_list = ["PLT_q", "Hb_q", "WBC_q", "Neut_q", "Lymph_q"]

# probability calibrated（method="isotonic")
clf_quartile = GaussianNB(priors = [0.5, 0.5])  
clf_quartile_isotonic = CalibratedClassifierCV(clf_quartile, cv=2, method="isotonic")
clf_quartile_isotonic_fit = clf_quartile_isotonic.fit(data_train_smo[quartile_list], data_train_smo["event"])

In [20]:
def predict_by_model_cal_iso(df):
    df["prior"] = clf_prior_isotonic_fit.predict_proba(df[prior_feature_list])[:,1]
    df["prior_odds"] = df["prior"]/(1 - df["prior"])

    quartile_pre = clf_quartile_isotonic_fit.predict_proba(df[quartile_list])[:,1]
    df["quartile_odds"] = quartile_pre/(1- quartile_pre)
    df["posterior_odds"] = df["quartile_odds"]*df["prior_odds"]
    df["posterior"] = df["posterior_odds"]/(1+df["posterior_odds"])  

    return df

In [21]:
data_train_smo  = predict_by_model_cal_iso(data_train_smo)

fpr, tpr, thresholds = roc_curve(data_train_smo["event"], data_train_smo["posterior"])
print(len(data_train_smo["event"]))
print(auc(fpr, tpr))

# data_train_smo.to_csv("./output/smote_model_smote_trainset_with_prediction_callibration_isotonic.csv", index=False)

3870
0.8269149156367472


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_isotonic_fit.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:7: SettingWithCopyWarning: 
A value is trying to be

In [22]:
data_test_smo  = predict_by_model_cal_iso(data_test_smo)

print(len(data_test_smo["event"]))
fpr, tpr, thresholds = roc_curve(data_test_smo["event"], data_test_smo["posterior"])
print(auc(fpr, tpr))

# data_test_smo.to_csv("./output/smote_model_smote_testset_with_prediction_callibration_isotonic.csv", index=False)

790
0.8739464829354271


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_isotonic_fit.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:7: SettingWithCopyWarning: 
A value is trying to be

# model performance for selected features v.s. all features

In [23]:
def calc_auc(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    return auc(fpr, tpr)

In [24]:
quartile_list = ["PLT_q", "Hb_q", "WBC_q", "Neut_q", "Lymph_q"]

auc_result_list = list()

for length in [4, 8]:
    data_train_smo_sub_length = data_train_smo[data_train_smo['cycle'] <= length]
    data_test_smo_sub_length = data_test_smo[data_test_smo['cycle'] <= length]
    
   
    df = data_test_smo_sub_length.copy()
    df['posterior'] = df['prior']
    # df.to_csv(f"./output/isotonic/testset_static_bayes_cycle_{length}_prediction_result.csv", index=False)
    auc_result_list.append({
        "model_name": "static_factors", 
        "followup_length": length,
        "auc": calc_auc(data_test_smo_sub_length['event'], data_test_smo_sub_length['prior'])
    })

    for dynamic_feature in quartile_list:
        clf_quartile_each_feature = GaussianNB(priors = [0.5, 0.5])
        clf_quartile_each_feature_isotonic = CalibratedClassifierCV(clf_quartile_each_feature, cv=2, method="isotonic")
        clf_quartile_each_feature_isotonic_fit = clf_quartile_each_feature_isotonic.fit(data_train_smo_sub_length[[dynamic_feature]], data_train_smo_sub_length["event"])
        
        df = data_test_smo_sub_length.copy()
        df["posterior"] = clf_quartile_each_feature_isotonic_fit.predict_proba(df[[dynamic_feature]])[:,1]
        
        # df.to_csv(f"./output/isotonic/testset_{dynamic_feature}_cycle_{length}_model_prediction_result.csv", index=False)
        auc_result_list.append({
            "model_name": dynamic_feature, 
            "followup_length": length,
            "auc": calc_auc(df['event'], df['posterior'])
        })

    # data_test_smo_sub_length.to_csv(f"./output/isotonic/testset_dynamic_plus_static_bayes_cycle_{length}_prediction_result.csv", index=False)
    auc_result_list.append({
        "model_name": "static+dynamic", 
        "followup_length": length,
        "auc": calc_auc(data_test_smo_sub_length['event'], data_test_smo_sub_length['posterior'])
    })

# pd.DataFrame(auc_result_list).to_csv("./output/isotonic/auc_for_different_model.csv", index = False)

In [25]:
auc_result_list

[{'model_name': 'static_factors',
  'followup_length': 4,
  'auc': 0.7309407422186752},
 {'model_name': 'PLT_q', 'followup_length': 4, 'auc': 0.6956903431763768},
 {'model_name': 'Hb_q', 'followup_length': 4, 'auc': 0.6794293695131683},
 {'model_name': 'WBC_q', 'followup_length': 4, 'auc': 0.8000349162011173},
 {'model_name': 'Neut_q', 'followup_length': 4, 'auc': 0.829938148443735},
 {'model_name': 'Lymph_q', 'followup_length': 4, 'auc': 0.6897595770151637},
 {'model_name': 'static+dynamic',
  'followup_length': 4,
  'auc': 0.878396847565842},
 {'model_name': 'static_factors',
  'followup_length': 8,
  'auc': 0.7544015382150295},
 {'model_name': 'PLT_q', 'followup_length': 8, 'auc': 0.6486107995513539},
 {'model_name': 'Hb_q', 'followup_length': 8, 'auc': 0.6849126742509213},
 {'model_name': 'WBC_q', 'followup_length': 8, 'auc': 0.7869091491748119},
 {'model_name': 'Neut_q', 'followup_length': 8, 'auc': 0.8026181701650377},
 {'model_name': 'Lymph_q', 'followup_length': 8, 'auc': 0.682

# model performance on raw data

In [27]:
data_raw = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/output/data_valid.csv")

data_valid = data_raw
i = 4 # 8
data_valid = data_valid[data_valid.cycle <= i]

data_train = data_valid[data_valid["visit_year"] <= 2018]
X_train = data_train[[x for x in data.columns if x not in ["patient_sn", "visit_year", "cycle_num", "event"]]]
y_train = data_train["event"]

data_test = data_valid[data_valid["visit_year"] > 2018]
data_test.reset_index(inplace = True)
X_test = data_test[[x for x in data.columns if x not in ["patient_sn", "visit_year", "cycle_num", "event"]]]
y_test = data_test["event"]

In [30]:
data_train  = predict_by_model_cal_iso(data_train) 
# data_train.to_csv("./output/smote_model_on_raw_trainset_with_prediction_isotonic.csv", index=False)

data_test_sub = data_test[data_test["cycle"]<=4] 
data_test  = predict_by_model_cal_iso(data_test)
# data_test.to_csv("./output/smote_model_on_raw_testset_with_prediction_isotonic.csv", index=False)

fpr, tpr, thresholds = roc_curve(data_test_sub["event"], data_test_sub["posterior"])
print(auc(fpr, tpr))

0.7735119047619047


/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior"] = clf_prior_isotonic_fit.predict_proba(df[prior_feature_list])[:,1]
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prior_odds"] = df["prior"]/(1 - df["prior"])
/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/324101349.py:7: SettingWithCopyWarning: 
A value is trying to be

# logistic regression of same variables with Bayes (baseline)—SMO data

In [34]:
order_dynamic_feature_list= ["PLT", "Hb", "WBC", "Neut", "Lymph"] 
prior_feature_list = ["gender", "tumor_stage_pN", "cycle"]

In [38]:
auc_result_list = list()
for i in range(len(order_dynamic_feature_list), len(order_dynamic_feature_list)+1):
    auc_result_dict = dict()
    auc_result_dict["index"] = i
    select_dynamic_feature = order_dynamic_feature_list[:i]
    select_feature = select_dynamic_feature + prior_feature_list
    
    lr = Logit(data_train_smo["event"], add_constant(data_train_smo[select_feature], prepend = False)).fit_regularized(method = "l1")
    print(lr.summary())
    
    # 训练集
#     y_pre = lr.predict(add_constant(data_train_smo[select_feature], prepend = False))
#     fpr, tpr, thresholds = roc_curve(data_train_smo["event"], y_pre)
#     auc_result_dict["data_train auc"] = auc(fpr, tpr)
#     data_train_smo["predict"]  = lr.predict(add_constant(data_train_smo[select_feature], prepend = False))
#     data_train_smo.to_csv("./output/log_model_on_smo_trainset_with_prediction_new.csv", index=False)
    
    # 测试集
    y_pre = lr.predict(add_constant(data_test_smo[select_feature], prepend = False))
    fpr, tpr, thresholds = roc_curve(data_test_smo["event"], y_pre)
    auc_result_dict["data_test auc"] = auc(fpr, tpr)
    data_test_smo["predict"]  = lr.predict(add_constant(data_test_smo[select_feature], prepend = False))
#     data_test_smo.to_csv("./output/log_model_on_smo_testset_with_prediction_new.csv", index=False)
    auc_result_list.append(auc_result_dict)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5467086811270958
            Iterations: 41
            Function evaluations: 42
            Gradient evaluations: 41
                           Logit Regression Results                           
Dep. Variable:                  event   No. Observations:                 3870
Model:                          Logit   Df Residuals:                     3861
Method:                           MLE   Df Model:                            8
Date:                Fri, 17 Jun 2022   Pseudo R-squ.:                  0.2113
Time:                        18:29:32   Log-Likelihood:                -2115.8
converged:                       True   LL-Null:                       -2682.5
Covariance Type:            nonrobust   LLR p-value:                2.302e-239
                     coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------

/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/1333492688.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_smo["predict"]  = lr.predict(add_constant(data_train_smo[select_feature], prepend = False))


In [39]:
pd.DataFrame(auc_result_list).sort_values(by = "data_train auc", ascending=False).head(10)

,index,data_train auc
0,5,0.795303


In [37]:
pd.DataFrame(auc_result_list).sort_values(by = "data_test auc", ascending=False).head(10)

,index,data_test auc
0,5,0.801093


# logistic regression of same variables with Bayes (baseline)—raw data

In [47]:
order_dynamic_feature_list= ["PLT", "Hb", "WBC", "Neut", "Lymph"]
prior_feature_list = ["gender", "tumor_stage_pN", "cycle"]

In [48]:
auc_result_list = list()
for i in range(len(order_dynamic_feature_list), len(order_dynamic_feature_list)+1):
    auc_result_dict = dict()
    auc_result_dict["index"] = i
    select_dynamic_feature = order_dynamic_feature_list[:i]
    select_feature = select_dynamic_feature + prior_feature_list
    
    lr = Logit(data_train["event"], add_constant(data_train[select_feature], prepend = False)).fit_regularized(method = "l1")
    print(lr.summary())
    
    # 训练集
    y_pre = lr.predict(add_constant(data_train[select_feature], prepend = False))
    fpr, tpr, thresholds = roc_curve(data_train["event"], y_pre)
    auc_result_dict["data_train auc"] = auc(fpr, tpr)
    data_train["predict"]  = lr.predict(add_constant(data_train[select_feature], prepend = False))
#     data_train.to_csv("./output/log_model_on_raw_trainset_with_prediction_new.csv", index=False)  
   
    # 测试集
#     y_pre = lr.predict(add_constant(data_test[select_feature], prepend = False))
#     fpr, tpr, thresholds = roc_curve(data_test["event"], y_pre)
#     auc_result_dict["data_test auc"] = auc(fpr, tpr)
#     data_test["predict"]  = lr.predict(add_constant(data_test[select_feature], prepend = False))
#     data_test.to_csv("./output/log_model_on_raw_testset_with_prediction_new.csv", index=False)
    auc_result_list.append(auc_result_dict)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.21008401274150418
            Iterations: 59
            Function evaluations: 61
            Gradient evaluations: 59
                           Logit Regression Results                           
Dep. Variable:                  event   No. Observations:                 2057
Model:                          Logit   Df Residuals:                     2048
Method:                           MLE   Df Model:                            8
Date:                Fri, 17 Jun 2022   Pseudo R-squ.:                 0.06656
Time:                        18:34:06   Log-Likelihood:                -432.14
converged:                       True   LL-Null:                       -462.96
Covariance Type:            nonrobust   LLR p-value:                 2.233e-10
                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------

/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_96395/94055453.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train["predict"]  = lr.predict(add_constant(data_train[select_feature], prepend = False))


In [49]:
pd.DataFrame(auc_result_list).sort_values(by = "data_train auc", ascending=False).head(10)

,index,data_train auc
0,5,0.711217


In [46]:
pd.DataFrame(auc_result_list).sort_values(by = "data_test auc", ascending=False).head(10)

,index,data_test auc
0,5,0.583122
